# Imports

In [13]:
from collections import defaultdict

from gensim.corpora import BleiCorpus, Dictionary
from gensim.models import LdaModel
from gensim.matutils import argsort,sparse2full

import json

import matplotlib
from matplotlib import pyplot as plt

import numpy as np
import networkx as nx

import os

import pickle

from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist, squareform
from scipy.stats import gaussian_kde

import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import ShuffleSplit
from sklearn.linear_model import RandomizedLasso
from sklearn.metrics import r2_score, f1_score
from sklearn.preprocessing import LabelEncoder

from statsmodels.nonparametric.kde import KDEUnivariate

import wiki

matplotlib.style.use("ggplot")

# Loading the data

In [2]:
all_corpora = json.load(open("all_corpora.json"))

In [3]:
id2word = Dictionary.load("all_corpora.dict")
model = LdaModel.load("lda_models/lda_all_corpora_150topics_50passes")
model.dictionary = id2word

In [4]:
topic_colors = sns.color_palette("hls", model.num_topics)

In [5]:
all_corpora = {
    name: id2word.doc2bow(doc) 
    for name, doc in all_corpora.items()
}

In [6]:
# Filter noise
noise_topics = [24, 35, 53, 67, 88, 102, 115, 129, 139]

# Splitting corpora

In [7]:
jpam_corpus = {
    name: bow
    for name, bow in all_corpora.items()
    if "all_texts/jpam_texts/" in name
}

jpart_corpus = {
    name: bow
    for name, bow in all_corpora.items()
        if "all_texts/jpart_texts/" in name
}

par_corpus = {
    name: bow
    for name, bow in all_corpora.items()
    if "all_texts/par_texts/" in name
}

In [10]:
jpam_doi_to_vec = {
    os.path.splitext(os.path.split(name)[1])[0]: sparse2full(model[bow], model.num_topics)
    for name, bow in jpam_corpus.items()
}

In [14]:
jpam_doi_to_vec = {
    doi: vec.tolist()
    for doi, vec in jpam_doi_to_vec.items()
}

json.dump(jpam_doi_to_vec, open("jpam_doi_to_150vec.json", "w"))

# Nearest Neighbors

In [8]:
K = 3

In [9]:
jpam_vectors = {
    name: sparse2full(model[bow], model.num_topics)
    for name, bow in jpam_corpus.items()
}

jpart_vectors = {
    name: sparse2full(model[bow], model.num_topics)
    for name, bow in jpart_corpus.items()
}

par_vectors = {
    name: sparse2full(model[bow], model.num_topics)
    for name, bow in par_corpus.items()
}

In [10]:
jpam_names, jpam_vecs = zip(*jpam_vectors.items())
jpart_names, jpart_vecs = zip(*jpart_vectors.items())
par_names, par_vecs = zip(*par_vectors.items())

In [11]:
all_names = list(jpam_names) + list(jpart_names) + list(par_names)
all_vecs = list(jpam_vecs) + list(jpart_vecs) + list(par_vecs)

In [12]:
all_vecs = np.array(all_vecs)
paper_distances = pdist(all_vecs, metric="cosine")
paper_distances = squareform(paper_distances)

### Fixed count

In [164]:
paper_numeric = paper_distances.argsort()[:, :K]
paper_neighbors = paper_numeric

In [165]:
GNN = nx.Graph()
jpam_nodes = [i for i, name in enumerate(all_names) if name in jpam_names]
jpart_nodes = [i for i, name in enumerate(all_names) if name in jpart_names]
par_nodes = [i for i, name in enumerate(all_names) if name in par_names]
for neighborhood in paper_numeric:
    n1 = neighborhood[0]
    for i in range(1, K):
        n2 = neighborhood[i]
        GNN.add_edge(n1, n2, weight=paper_distances[n1][n2])

In [166]:
pos = nx.spring_layout(GNN)

In [ ]:
paper_neighbors

In [167]:
fig, ax = plt.subplots(figsize=(50, 50))
nx.draw_networkx_nodes(GNN, pos, nodelist=jpam_nodes, node_color='b', alpha=0.5)
nx.draw_networkx_nodes(GNN, pos, nodelist=jpart_nodes, node_color='r', alpha=0.5)
nx.draw_networkx_nodes(GNN, pos, nodelist=par_nodes, node_color='g', alpha=0.5)
nx.draw_networkx_edges(GNN, pos)
plt.suptitle("Papers linked by ")
plt.savefig('paper_distances.png')

### Threshold

In [168]:
edge_threshold = 0.15

In [169]:
GTN = nx.from_numpy_matrix(paper_distances)

In [170]:
orig_edges = GTN.edges()

In [171]:
orig_nodes = GTN.nodes()

In [172]:
GTN.remove_edges_from([
    edge for edge in GTN.edges()
    if GTN.get_edge_data(*edge)['weight'] > edge_threshold
])
GTN.remove_nodes_from([
    node for node in GTN.nodes()
    if len(GTN.neighbors(node)) == 0
])
fjpam_nodes = [n for n in jpam_nodes if GTN.has_node(n)]
fjpart_nodes = [n for n in jpart_nodes if GTN.has_node(n)]
fpar_nodes = [n for n in par_nodes if GTN.has_node(n)]

In [173]:
pos_thresh = nx.spring_layout(GTN)

In [195]:
fig, ax = plt.subplots(figsize=(50, 50))
edge_weights = [GTN.get_edge_data(*edge)["weight"] for edge in GTN.edges()]
max_edge_weight = max(edge_weights)
min_edge_weight = min(edge_weights)
range_edge_weight = max_edge_weight - min_edge_weight
edge_colors = [
    ((max_edge_weight - GTN.get_edge_data(*edge)["weight"])/ range_edge_weight,) * 3
    for edge in GTN.edges()
]
nodes_by_topic = defaultdict(list)
for node in GTN.nodes():
    vec = all_vecs[node]
    top_topic = vec.argsort()[-1]
    nodes_by_topic[top_topic].append(node)

for topic_num in range(model.num_topics):
    nx.draw_networkx_nodes(GTN, pos_thresh, nodelist=nodes_by_topic[topic_num], alpha=0.75, node_color=topic_colors[topic_num])
# nx.draw_networkx_nodes(GTN, pos_thresh, nodelist=fjpam_nodes, alpha=0.75, node_color='b')
# nx.draw_networkx_nodes(GTN, pos_thresh, nodelist=fjpart_nodes, alpha=0.75, node_color='r')
# nx.draw_networkx_nodes(GTN, pos_thresh, nodelist=fpar_nodes, alpha=0.75, node_color='k')
nx.draw_networkx_edges(GTN, pos_thresh, edge_color=edge_colors) 
ax.grid(b=False)
ax.margins(0)
fig.suptitle("Papers linked by close neighbors, colored by topic", fontsize=48)
fig.tight_layout()
plt.savefig('paper_dist_thresh_tcolor.png')

# Kernel Densities

In [176]:
all_vecs[:, 0]

array([ 0.        ,  0.        ,  0.05433092, ...,  0.        ,
        0.        ,  0.        ], dtype=float32)

In [179]:
fig, axes = plt.subplots((model.num_topics // 4) + 1, 4, figsize=(60, 100), facecolor="w", edgecolor="k")
# fig.set_size_inches(12, 16)
fig.subplots_adjust(hspace=0.5, wspace=.1)
axes = axes.ravel()
for topic_num in range(model.num_topics):
    ax = axes[topic_num]
    ax.hist(all_vecs[:, topic_num], bins=20)
plt.savefig("kde.png")
# plt.show()

# Dendrogram + Continualizing Topics

In [13]:
topic_term = model.state.get_lambda()
distances = pdist(topic_term, metric='cosine')
Z = linkage(distances, "ward")

fig = plt.figure(figsize=(20, 40))
plt.title("Topic Dendrogram")
plt.xlabel("Distance")

dendrogram(
    Z,
    orientation="left"
)

ax = fig.get_axes()[0]
topic_ordering = [int(item.get_text()) for item in ax.get_yticklabels()]

In [15]:
jpam_avg_dist = np.average(jpam_vecs, axis=0)

jpart_avg_dist = np.average(jpart_vecs, axis=0)

par_avg_dist = np.average(par_vecs, axis=0)

In [19]:
# Copy
f_jpam_avg_dist = jpam_avg_dist.copy()
f_jpart_avg_dist = jpart_avg_dist.copy()
f_par_avg_dist = par_avg_dist.copy()
# Remove
f_jpam_avg_dist[noise_topics] = 0
f_jpart_avg_dist[noise_topics] = 0
f_par_avg_dist[noise_topics] = 0

In [20]:
top_jpam_topics = argsort(f_jpam_avg_dist, topn=15, reverse=True)
top_jpart_topics = argsort(f_jpart_avg_dist, topn=15, reverse=True)
top_par_topics = argsort(f_par_avg_dist, topn=15, reverse=True)

In [51]:
def write_top_topics(fp, top_topics):
    for tid in top_topics:
        terms = [term for term, score in model.show_topic(tid, topn=3)]
        print(terms)
        labels = wiki.page_labels(terms, 1)
        topic_summary = model.print_topic(tid, topn=3)
        w.write("\t%03d : %s : %s\n"  % (tid, repr(labels), topic_summary))

with open("top_topic_summary.txt", "w") as w:
    w.write("JPAM TOP TOPICS:\n")
    write_top_topics(w, top_jpam_topics)
    w.write("\nJPART TOP TOPICS:\n")
    write_top_topics(w, top_jpam_topics)
    w.write("\nPAR TOP TOPICS:\n")
    write_top_topics(w, top_par_topics)

['variable', 'table', 'sample']
['child', 'family', 'welfare']
['incentive', 'efficiency', 'benefit']
['award', 'sustainability', 'environmental']
['student', 'grade', 'score']
['estimate', 'bias', 'statistical']
['employment', 'labor', 'worker']
['poverty', 'income', 'poor']
['ethic', 'democratic', 'moral']
['country', 'global', 'nation']
['tax', 'revenue', 'income']
['analyst', 'maker', 'criterion']
['institutional', 'manager', 'actor']
['housing', 'neighborhood', 'urban']
['learning', 'culture', 'leadership']
['variable', 'table', 'sample']
['child', 'family', 'welfare']
['incentive', 'efficiency', 'benefit']
['award', 'sustainability', 'environmental']
['student', 'grade', 'score']
['estimate', 'bias', 'statistical']
['employment', 'labor', 'worker']
['poverty', 'income', 'poor']
['ethic', 'democratic', 'moral']
['country', 'global', 'nation']
['tax', 'revenue', 'income']
['analyst', 'maker', 'criterion']
['institutional', 'manager', 'actor']
['housing', 'neighborhood', 'urban']
['

In [32]:
for tid in top_jpart_topics:
    terms = [term for term, score in model.show_topic(tid, topn=5)]
    print("%03d" % tid, ":", wiki.page_labels(terms, 1), model.print_topic(tid, topn=3))

012 : ['Institutional racism'] 0.016*"institutional" + 0.007*"manager" + 0.006*"actor"
134 : ['Bureaucratic drift'] 0.032*"bureaucratic" + 0.029*"bureaucracy" + 0.018*"principal"
121 : ['Customer satisfaction'] 0.043*"survey" + 0.033*"respondent" + 0.024*"variable"
142 : ['Nel Noddings'] 0.018*"ethic" + 0.008*"democratic" + 0.008*"moral"
043 : ['Collaborative partnership'] 0.111*"network" + 0.032*"collaboration" + 0.032*"collaborative"
086 : ['Leadership'] 0.021*"learning" + 0.013*"culture" + 0.012*"leadership"
047 : ['Capital structure'] 0.033*"manager" + 0.026*"managerial" + 0.024*"variable"
073 : ['Categorical variable'] 0.043*"inﬂuence" + 0.037*"signiﬁcant" + 0.027*"ﬁndings"
030 : ['George Washington University Graduate School of Education and Human Development'] 0.016*"affair" + 0.012*"graduate" + 0.008*"george"
037 : ['Sample size determination'] 0.033*"variable" + 0.029*"table" + 0.020*"sample"
108 : ['Public service motivation'] 0.054*"motivation" + 0.038*"psm" + 0.019*"commitm

In [35]:
for tid in top_par_topics:
    terms = [term for term, score in model.show_topic(tid, topn=3)]
    print("%03d" % tid, ":", wiki.page_labels(terms, 2), model.print_topic(tid, topn=3))

142 : ['Democratic capitalism', 'Consistent life ethic'] 0.018*"ethic" + 0.008*"democratic" + 0.008*"moral"
086 : ['Service-learning', 'Learning organization'] 0.021*"learning" + 0.013*"culture" + 0.012*"leadership"
090 : ['Effects-based operations', 'Hurricane Katrina effects by region'] 0.044*"diﬀerent" + 0.033*"oﬃcials" + 0.027*"eﬀects"
012 : ['Environmental manager', 'Institutional racism'] 0.016*"institutional" + 0.007*"manager" + 0.006*"actor"
077 : ["Standing Committee of the National People's Congress", 'United States Congress Joint Committee on Taxation'] 0.023*"committee" + 0.019*"congress" + 0.016*"commission"
070 : ['Global Peace Index', 'Island country'] 0.054*"country" + 0.020*"global" + 0.018*"nation"
126 : ['Participation (decision making)', 'Participatory democracy'] 0.111*"citizen" + 0.066*"participation" + 0.021*"democracy"
121 : ['Survey methodology', 'Survey (human research)'] 0.043*"survey" + 0.033*"respondent" + 0.024*"variable"
043 : ['Collaboration', 'Collabora

# Fingerprints

In [241]:
plt.close('all')

In [245]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(20, 10))
y_pos = topic_ordering

# Reorder to be more continuous
f_jpam_avg_dist = f_jpam_avg_dist[topic_ordering]
f_jpart_avg_dist = f_jpart_avg_dist[topic_ordering]
f_par_avg_dist = f_par_avg_dist[topic_ordering]

ax1.barh(np.arange(150), f_jpam_avg_dist)
ax1.set_title("JPAM")
ax2.barh(np.arange(150), f_jpart_avg_dist)
ax2.set_title("JPART")
ax3.barh(np.arange(150), f_par_avg_dist)
ax3.set_title("PAR")

for ax in (ax1, ax2, ax3):
    ax.set_ybound(lower=0, upper=150)
    ax.set_xbound(lower=0, upper=0.055)
fig.savefig("fingerprints_new.png")

In [237]:
fig, ax = plt.subplots(figsize=(30, 15))
width = .75
N = model.num_topics

diff_jpam_par = f_jpam_avg_dist - f_par_avg_dist
diff_jpam_jpart = f_jpam_avg_dist - f_jpart_avg_dist

mask1 = np.ma.where(diff_jpam_par >= diff_jpam_jpart)
mask2 = np.ma.where(diff_jpam_jpart >= diff_jpam_par)


ax.bar(np.arange(N)[mask1], diff_jpam_par[mask1], width=width, align="edge", alpha=1, color='darkorange', label="JPAM--PAR")
ax.bar(np.arange(N), diff_jpam_jpart, width=width, align="edge", alpha=1, color='b', label="JPAM--JPART")
ax.bar(np.arange(N)[mask2], diff_jpam_par[mask2], width=width, align="edge", alpha=1, color='darkorange')

x_ticks = ax.set_xticks(np.arange(N))
x_tick_labels = ax.set_xticklabels(topic_ordering, rotation="vertical")
ax.set_title("JPAM TOPICS - PAR/JPART TOPICS")

ax.grid(b=False)
ax.legend()

fig.savefig("topic_diff_new.png")

# Feature Selection

In [25]:
journal_names = []
for name in all_names:
    if "jpam_texts" in name:
        journal_names.append("JPAM")
    if "jpart_texts" in name:
        journal_names.append("JPART")
    if "par_texts" in name:
        journal_names.append("PAR")
        
topic_names = ["Topic %d" % i for i in range(model.num_topics)]

f_all_vecs = np.array(all_vecs).copy()
f_all_vecs[:, noise_topics] = 0
X = f_all_vecs
y = np.array(journal_names)

rf = RandomForestClassifier(max_features=150, n_estimators=20)
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=150, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

### Native Feature importances

In [26]:
feature_importances = sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), topic_names), reverse=True)

### F1 Drop with random permutations

In [54]:
scores = defaultdict(list)
for train_idx, test_idx in ShuffleSplit(len(X), 100, 0.3):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    r = rf.fit(X_train, y_train)
    acc = f1_score(y_test, r.predict(X_test), average='micro')
    for i in range(X.shape[1]):
        X_t = X_test.copy()
        np.random.shuffle(X_t[:, i])
        shuffle_acc = f1_score(y_test, r.predict(X_t), average='micro')
        scores[topic_names[i]].append((acc-shuffle_acc) / acc)

feature_acc_drop = sorted([
    (round(np.mean(score), 4), feat)
    for feat, score in scores.items()
], reverse=True)

### Stability Selection

In [43]:
rlasso = RandomizedLasso(alpha=0.0005)
le = LabelEncoder()
rlasso.fit(X, le.fit_transform(y))

stability_selected = sorted(zip(rlasso.scores_, range(model.num_topics)), reverse=True)

In [52]:
most_stable_topics = [tid for score, tid in stability_selected]
with open("discriminatory_topics.txt", "w") as w:
    w.write("MOST DISCRIMINATORY TOPICS:\n")
    write_top_topics(w, most_stable_topics[:10])

['ethic', 'democratic', 'moral']
['child', 'family', 'welfare']
['diﬀerent', 'oﬃcials', 'eﬀects']
['learning', 'culture', 'leadership']
['variable', 'table', 'sample']
['scholar', 'speciﬁc', 'ﬁeld']
['committee', 'congress', 'commission']
['survey', 'respondent', 'variable']
['student', 'curriculum', 'faculty']
['employment', 'labor', 'worker']


In [53]:
feature_acc_drop[:10]

NameError: name 'feature_acc_drop' is not defined

In [65]:
feature_importances[:10]

[(0.3044, 'Topic 90'),
 (0.088599999999999998, 'Topic 67'),
 (0.075800000000000006, 'Topic 73'),
 (0.043499999999999997, 'Topic 30'),
 (0.0292, 'Topic 86'),
 (0.025899999999999999, 'Topic 12'),
 (0.0253, 'Topic 142'),
 (0.0241, 'Topic 134'),
 (0.015100000000000001, 'Topic 83'),
 (0.013599999999999999, 'Topic 77')]

In [95]:
model.print_topic(114)

'0.061*"employment" + 0.060*"labor" + 0.050*"worker" + 0.039*"wage" + 0.017*"employer" + 0.012*"unemployment" + 0.008*"skill" + 0.008*"credit" + 0.008*"hiring" + 0.007*"minimum"'

# Random Stuff

In [111]:
topn=10
top_fjpam_topics = argsort(f_jpam_avg_dist, topn=topn, reverse=True)
top_fjpart_topics = argsort(f_jpart_avg_dist, topn=topn, reverse=True)
top_fpar_topics = argsort(f_par_avg_dist, topn=topn, reverse=True)

In [112]:
for tid in top_fjpam_topics:
    print("%03d" % tid, ":", f_jpam_avg_dist[tid], ":", model.print_topic(tid, topn=5))

print()
print("-" * 80)
print()
    
for tid in top_fjpart_topics:
    print("%03d" % tid, ":", f_jpart_avg_dist[tid], ":", model.print_topic(tid, topn=5))
    
print()
print("-" * 80)
print()
    
for tid in top_fpar_topics:
    print("%03d" % tid, ":", f_par_avg_dist[tid], ":", model.print_topic(tid, topn=5))

037 : 0.0410403 : 0.033*"variable" + 0.029*"table" + 0.020*"sample" + 0.019*"estimate" + 0.018*"significant"
118 : 0.0406323 : 0.081*"child" + 0.050*"family" + 0.044*"welfare" + 0.017*"income" + 0.017*"parent"
018 : 0.023166 : 0.025*"incentive" + 0.018*"efficiency" + 0.015*"benefit" + 0.013*"price" + 0.007*"output"
034 : 0.020855 : 0.034*"award" + 0.027*"sustainability" + 0.020*"environmental" + 0.014*"conference" + 0.014*"sustainable"
082 : 0.0208324 : 0.100*"student" + 0.029*"grade" + 0.028*"score" + 0.019*"test" + 0.019*"charter"
022 : 0.0183188 : 0.026*"estimate" + 0.017*"bias" + 0.011*"statistical" + 0.011*"score" + 0.010*"comparison"
114 : 0.0177116 : 0.061*"employment" + 0.060*"labor" + 0.050*"worker" + 0.039*"wage" + 0.017*"employer"
092 : 0.0163509 : 0.088*"poverty" + 0.055*"income" + 0.015*"poor" + 0.013*"inequality" + 0.010*"family"
142 : 0.0151084 : 0.018*"ethic" + 0.008*"democratic" + 0.008*"moral" + 0.008*"administrator" + 0.008*"democracy"
070 : 0.0138542 : 0.054*"countr